# 1. 순차데이터와 RNN의 기본구조

## 1. 순차데이터의 특징 3가지는?

정답: 

## 2. RNN이 시간성, 가변길이, 문맥의존성을 모두 처리하기 위해 은닉층이 가지는 연결로, t-1 순간에 발생한 정보를 t순간으로 전달 가능케 하는 것은?

정답: 

# 2. PyTorch RNN(빈칸 채우기문제)
복습과제 폴더에 포함되어 같이 제공되는 PyTorch_RNN_과제.ipynb를 참고해주세요.

## 3. BPTT 개념문제
1. RNN은 순환 구조를 띄는 신경망이기 때문에 하단의 그림에서 〖𝑑ℎ〗(𝑡−1) 은 t-1 시점의 Loss에서 흘러들어온 그래디언트 뿐 아니라 ⭐️에 해당하는 그래디언트가 더해집니다. ⭐️이 의미하는 것을 수식으로 표현해주세요. (힌트 : h(t-1)= *)

**사진이 잘 안보인다면 발표자료pdf 32쪽 그림을 참고해주세요!
<img src='p1.png' width='400px' height='300px'></img>

답 : 

2. BPTT와 Truncated BPTT의 단점을 설명하고 이를 보완하는 방법을 설명해주세요.

답: 

## 4.  RNN을 사용해 주식가격의 변동 예측하기

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, SimpleRNN
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential

In [ ]:
# 데이터 불러오기
samsung = pd.read_csv("samsung.csv", index_col=0)
samsung

In [ ]:
samsung.info()

In [ ]:
samsung.describe()

In [ ]:
samsung.isnull().sum()

In [ ]:
high_prices = samsung['High'].values
low_prices = samsung['Low'].values
mid_prices = (high_prices+low_prices)/2
mid_prices

#중간 값 요소 추가하기
samsung['Mid'] = mid_prices
samsung

#Close 제거
samsung.drop("Close", axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(samsung['Adj Close'])
plt.title("Stock Price of Samsung")
plt.show()

In [ ]:
#데이터 스케일링 

samsung_minimum = samsung['Adj Close'].min()
samsung_maximum = samsung['Adj Close'].max()

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
samsung_scaled = scaler.fit_transform(samsung)
samsung = pd.DataFrame(samsung_scaled, columns=samsung.columns, index=list(samsung.index.values))
samsung

In [ ]:
# train/test set 나누기
# 앞에서부터 90%의 데이터를 train set으로, 나머지 10%를 test set으로 사용
train_size = int(len(samsung)*0.9)
X_train = np.array(samsung.drop('Adj Close', axis=1)[:train_size])
y_train = np.array(samsung['Adj Close'][:train_size])

X_test =np.array(samsung.drop("Adj Close", axis=1)[train_size:])
y_test = np.array(samsung['Adj Close'][train_size:])

In [ ]:
print(f"shape of X_train: {X_train.shape}")
print(f"shape of y_train: {y_train.shape}")
print(f"shape of X_test: {X_test.shape}")
print(f"shape of y_test: {y_test.shape}")

### 문제4-1 아래와 같이 데이터의 구조를 바꿔주는 이유를 간단하게 설명해주세요. 만약, MLP를 사용해서 이 데이터를 분석하려고 한다면 데이터의 구조를 어떻게 해야 할까요

In [ ]:
# 구조 바꾸기
input_size = 5
X_train = X_train.reshape(X_train.shape[0], 1 , input_size)
X_test = X_test.reshape(X_test.shape[0],1, input_size)
print(f"shape of X_train: {X_train.shape}")
print(f"shape of X_test: {X_test.shape}")

데이터의 구조를 바꿔주는 이유: 

MLP를 사용할 시 데이터의 구조: 

### 문제 4-2 주어진 설명에 맞춰 빈 부분을 채워 모델 구조를 완성해 주세요

In [ ]:
# 모델 구조 설정
model = Sequential()
model.add(SimpleRNN((빈 칸을 채워주세요), input_shape=(빈칸을 채워주세요)) # units=1000으로 설정하고, 적절한 input_shape를 찾아 값을 넣어주세요.
model.add(Dense(1,activation='tanh')) 
model.add((빈칸을 채워주세요)) #Dropout 레이어를 추가해주세요. Dropout비율은 0.25로 설정해주세요

callbacks = [EarlyStopping(monitor = 'val_loss', (빈칸을 채워주세요)=5), #기준 값이 연속으로 5 번 이상 향상되지 않을 때 종료하도록 설정하세요
             ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', (빈칸을 채워주세요)=True)] # 이전보다 성능이 향상이 모델을 저장할 수 있도록 설정하세요.
model.compile(optimizer=(빈칸을 채워주세요), loss=(빈칸을 채워주세요), metrics=['mae', 'mse']) # optimizer를 sgd으로, loss를 mse로 설정해주세요.
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=500, batch_size=100, verbose=(빈칸을 채워주세요), #훈련과정이 출력되지 않도록 설정하세요
                    (빈칸을 채워주세요)=0.2,callbacks=callbacks)  # Train 데이터 셋의 20%를 validation 셋으로 사용하도록 설정하세요

In [ ]:
# 예측 결과 확인하기
predictions = model.predict(X_test)

predictions_unscaled = predictions * (samsung_maximum-samsung_minimum) +samsung_minimum
y_test_unscaled = y_test * (samsung_maximum-samsung_minimum) +samsung_minimum

plt.figure(figsize=(10,5))
plt.plot(y_test_unscaled, label='actual')
plt.plot(predictions_unscaled, label='Prediction')
plt.legend(loc='upper right')
plt.show()